# LDA model for full text analysis of Brazilian Administrative Decrees

## Imports

In [ ]:
!pip install pyLDAvis==3.2.1
!pip install spacy==2.2.4
!pip install https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-2.2.5/pt_core_news_sm-2.2.5.tar.gz
!pip install nltk==3.4.5

     |████████████████████████████████| 1.7 MB 11.2 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.1-py2.py3-none-any.whl size=136187 sha256=c3c7c4ee9f0adf6a63aa1d1eb6248b1ea19cafdcf8ff0cf62ca5d074f659c829
  Stored in directory: /root/.cache/pip/wheels/c6/ee/a6/7c17a63623f940dff0b9cbd7e48a27543f088fa55a7d2b62d0
Successfully built pyLDAvis
     |████████████████████████████████| 21.2 MB 1.5 MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-py3-none-any.whl size=21186282 sha256=061cb7fd5549951fdfe8e4aa0bfe869088914aae971edfdbd8996b22764a583f
  Stored in directory: /root/.cache/pip/wheels/c3/f9/0c/5c014a36941a00f5df5fc0756cb961d7c457a978e697a6ce3b
Successfully built pt-core-news-sm
     |████████████████████████████████| 1.5 MB 8.4 MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449921 sha256=2504a6018294c23714264a9d0966cc59bb298b8069c6fa3e43ba5a9c010ea938
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c656

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import json

import nltk
import gensim
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

import pyLDAvis.gensim
import pickle 
import pyLDAvis

import math
from matplotlib import pyplot as plt

import spacy
import io

import pt_core_news_sm
nlp = pt_core_news_sm.load()

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

from pprint import pprint

import logging
import itertools
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINT

In [ ]:
# Print main versions used:
print("pt_core_news_sm version:", pt_core_news_sm.__version__)
print("Spacy version:", spacy.__version__)
print("NLTK version:", nltk.__version__)
import sklearn
print("Scikit version:", sklearn.__version__)

pt_core_news_sm version: 2.2.5
Spacy version: 2.2.4
NLTK version: 3.4.5
Scikit version: 1.0.1


## Get database

In [ ]:
# This code was executed on Google Colab. Upload the file "radar_db" version 0.1 to your drive and the "frequency_stopwords_list.csv" file here. Both files are available on Kaggle.
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive')
database_file = '/content/gdrive/MyDrive/database - radar/radar_db' # replace with your drive link
stop_words_upload = files.upload()

Mounted at /content/gdrive


Saving frequency_stopwords_list.csv to frequency_stopwords_list.csv


## Preprocessing Functions

In [ ]:
def get_decretos():
    conn = sqlite3.connect(database_file)
    c = conn.cursor()
    c.execute("SELECT cod_ident_ato, ementa, full_text, referenda FROM Decretos")
    decretos = c.fetchall()
    decretos = [(decreto[0], str(decreto[1]) + " " + str(decreto[2]), decreto[3]) for decreto in decretos]
    conn.close()
    for i in range(len(decretos)):
        referenda_aux = [ref.strip() for ref in decretos[i][2].split(";")]
        decretos[i] = list(decretos[i])
        decretos[i][2] = referenda_aux
        decretos[i] = tuple(decretos[i])
    decretos = pd.DataFrame(decretos)
    decretos.columns = ['cod_ident_ato', 'full_text', 'referenda']
    return decretos

def preprocess_full(decretos_df):
  number_of_decrees = len(decretos_df['full_text'])
  breaking_punctuation = ['.', '!', '?', ';', ':']
  for j,decreto in enumerate(decretos_df['full_text']):
      print("Preprocessing", j+1, "/", number_of_decrees)
      text = ""
      
      decreto = decreto.replace("/", " ")
      decreto = decreto.replace("º", '')
      tokenized_document = word_tokenize(decreto)
      for i, word in enumerate(tokenized_document):
          
          if word.lower() == "art":               # lidando com os padrões de numeração de artigos definidos na observação.
              if tokenized_document[i+2].isnumeric() and tokenized_document[i+3] == ".":
                  tokenized_document[i+3] = ' '
      
          if(word.isalpha() or word.isnumeric()):
              text+=(word.lower()+" ")
          elif word in breaking_punctuation:
              text+=(" . ")                       # padroniza pontuações específicas em ponto final.
          elif word[0].isnumeric() and ("," not in word):
              word = word.replace(".", "")
              if(word.isnumeric()):
                  text+=(word+" ")                # lidando com números com pontuação no meio. Desconsidera números reais, comumente referentes à moeda.
      
      text = text.replace("art  .", " art ")
      text = text.replace("arts  .", " arts ")
      
      # remoção de assinaturas
      if(text.rfind(" república") != -1):
          signature = text.rfind(" república")
          if signature < len(text)/2:
              signature = len(text)
      
      decretos_df['full_text'].iloc[j] = text[:signature] + " república."
  return decretos_df

def removeComumStopWords(stopwords_file,texts, st= stopwords.words('portuguese')):
    frequency_stopwords=pd.read_csv(stopwords_file, sep=',')
    frequency_stopwords = frequency_stopwords['0']
    frequency_stopwords=frequency_stopwords.values.tolist()
    frequency_stopwords = set(frequency_stopwords)
    texts_r = []
    
    for t in texts:
        final_t = ""
        for w in t.split(" "):
            if w not in st and w!="." and w not in frequency_stopwords:
                final_t += w + " "
        texts_r.append(final_t)
    return texts_r

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out



In [ ]:
decretos = get_decretos() # get decress from data base
decretos = decretos.drop(columns = ['cod_ident_ato','referenda'], axis=1)
stop_words = pd.read_csv(io.BytesIO(stop_words_upload['frequency_stopwords_list.csv']), sep=',') # Set dataframe of stop_words
decretos

,full_text
0,Dispõe sobre o Comitê Consultivo de Nanotecnol...
1,"Altera o Decreto nº 8.642 , de 19 de janeiro ..."
2,Promulga o Acordo entre o Governo da República...
3,Promulga o Acordo entre o Governo da República...
4,Dispõe sobre o Comitê Interministerial de Tecn...
...,...
6836,"PROMULGA O PROTOCOLO AO TRATADO DE AMIZADE, CO..."
6837,PRORROGA A VIGÊNCIA DOS RESTOS A PAGAR QUE ESP...
6838,"ALTERA O DECRETO N. 2.889, DE 21/12/1998, QUE ..."
6839,"REVOGA O ART. 20 DO DECRETO N. 2.451, DE 05/01..."


## Text analysis

In [ ]:
full_text_preprocessed = preprocess_full(decretos)
full_text_preprocessed = full_text_preprocessed['full_text'].values.tolist()

A saída de streaming foi truncada nas últimas 5000 linhas.
Preprocessing 1842 / 6841
Preprocessing 1843 / 6841
Preprocessing 1844 / 6841
Preprocessing 1845 / 6841
Preprocessing 1846 / 6841
Preprocessing 1847 / 6841
Preprocessing 1848 / 6841
Preprocessing 1849 / 6841
Preprocessing 1850 / 6841
Preprocessing 1851 / 6841
Preprocessing 1852 / 6841
Preprocessing 1853 / 6841
Preprocessing 1854 / 6841
Preprocessing 1855 / 6841
Preprocessing 1856 / 6841
Preprocessing 1857 / 6841
Preprocessing 1858 / 6841
Preprocessing 1859 / 6841
Preprocessing 1860 / 6841
Preprocessing 1861 / 6841
Preprocessing 1862 / 6841
Preprocessing 1863 / 6841
Preprocessing 1864 / 6841
Preprocessing 1865 / 6841
Preprocessing 1866 / 6841
Preprocessing 1867 / 6841
Preprocessing 1868 / 6841
Preprocessing 1869 / 6841
Preprocessing 1870 / 6841
Preprocessing 1871 / 6841
Preprocessing 1872 / 6841
Preprocessing 1873 / 6841
Preprocessing 1874 / 6841
Preprocessing 1875 / 6841
Preprocessing 1876 / 6841
Preprocessing 1877 / 6841
Prepr

In [ ]:
full_text_processed = removeComumStopWords("frequency_stopwords_list.csv",full_text_preprocessed)
full_text_processed_docs = [doc.split(" ") for doc in full_text_processed]

In [ ]:
data_lemmatized_full = lemmatization(full_text_processed_docs, allowed_postags=['NOUN', 'ADJ'])
full_text_splited_doc = [doc.split(" ") for doc in data_lemmatized_full]

A saída de streaming foi truncada nas últimas 5000 linhas.
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarni

In [ ]:
dictionary = gensim.corpora.Dictionary(full_text_splited_doc)

non_filtering_full = dictionary
dictionary.filter_extremes(no_below=3, no_above=0.8)

bow_corpus_full = [dictionary.doc2bow(doc) for doc in full_text_splited_doc]

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : built Dictionary(12676 unique tokens: ['absoluto', 'alocação', 'ano', 'aperfeiçoamento', 'aprovação']...) from 6841 documents (total 1299370 corpus positions)
INFO : discarding 6212 tokens: [('atribuição', 6810), ('cbpf', 1), ('datar', 6714), ('decretar', 6830), ('nanotecnologia', 1), ('nanotecnologias', 2), ('7857', 1), ('ccl', 2), ('ccli', 2), ('ccxxv', 1)]...
INFO : keeping 6464 tokens which were in no less than 3 and no more than 5472 (=80.0%) documents
INFO : resulting dictionary: Dictionary(6464 unique tokens: ['absoluto', 'alocação', 'ano', 'aperfeiçoamento', 'aprovação']...)


## Validation

In [ ]:
vectorizer = CountVectorizer(analyzer='word', min_df=3, stop_words='english', lowercase=True, token_pattern='[a-zA-Z0-9]{3,}')
data_vectorized_full = vectorizer.fit_transform(data_lemmatized_full)

# Define Search Param
search_params = {'n_components': [5,10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized_full)

GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 learning_offset=50.0,
                                                 max_iter=5, random_state=0),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 10, 15, 20, 25, 30]})

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best params: ", model.best_params_)
print("Best Number of components: ", model.best_params_['n_components'])
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized_full))

Best params:  {'learning_decay': 0.5, 'n_components': 25}
Best Number of components:  25
Best Log Likelihood Score:  -1868031.8851055547
Model Perplexity:  564.2535896099004


## Final model creation

In [ ]:
lda_model_n = gensim.models.LdaModel(bow_corpus_full, num_topics=25, id2word=dictionary, passes=30, alpha='auto', random_state=42)

A saída de streaming foi truncada nas últimas 5000 linhas.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np

In [ ]:
lda_model_n.show_topics(num_topics=25)

[(0,
  '0.053*"avaliação" + 0.048*"servidor" + 0.028*"desempenhar" + 0.023*"cargo" + 0.019*"individual" + 0.016*"institucional" + 0.015*"exercício" + 0.014*"período" + 0.012*"resultar" + 0.012*"lei"'),
 (1,
  '0.051*"militar" + 0.029*"armar" + 0.025*"serviço" + 0.018*"oficial" + 0.011*"transportar" + 0.011*"público" + 0.010*"forçar" + 0.010*"idoso" + 0.010*"redação" + 0.009*"lei"'),
 (2,
  '0.023*"prazo" + 0.020*"lei" + 0.014*"jurídico" + 0.013*"pessoa" + 0.010*"decisão" + 0.010*"serviço" + 0.009*"entidade" + 0.009*"administrativo" + 0.009*"formar" + 0.008*"hipótese"'),
 (3,
  '0.041*"público" + 0.026*"serviço" + 0.024*"órgão" + 0.022*"entidade" + 0.020*"preço" + 0.014*"contratação" + 0.013*"contratar" + 0.011*"direta" + 0.009*"licitação" + 0.009*"proposto"'),
 (4,
  '0.232*"pontar" + 0.084*"linha" + 0.031*"seguir" + 0.028*"s" + 0.025*"indígena" + 0.024*"vinho" + 0.022*"zonar" + 0.021*"uva" + 0.018*"reta" + 0.016*"geográfico"'),
 (5,
  '0.036*"órgão" + 0.027*"atividades" + 0.021*"públi

## Final model visualization

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_n, bow_corpus_full, dictionary)
LDAvis_prepared

INFO : NumExpr defaulting to 2 threads.


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2     -0.125969 -0.092396       1        1  9.828595
17    -0.132694  0.112234       2        1  7.227263
13    -0.050266 -0.187998       3        1  6.880248
24    -0.138827 -0.105885       4        1  6.824471
21    -0.042154  0.261235       5        1  5.835186
5     -0.092121  0.226118       6        1  5.533626
23    -0.064774 -0.143207       7        1  5.172300
10    -0.121526 -0.039189       8        1  4.855870
20     0.275871 -0.023148       9        1  4.401443
22    -0.051617  0.212901      10        1  4.119797
3     -0.154236 -0.032578      11        1  4.055522
19     0.148172  0.135969      12        1  3.969753
0     -0.096161  0.007350      13        1  3.753482
6     -0.027686  0.016517      14        1  3.662929
15    -0.116925  0.021046      15        1  3.477608
8     -0.119992  0.013540      16        1  3.141295
11    -0.095140 -0.112247      17        1  3.062617
12     0.338969 -0.006991      18        1  2.932300
1     -0.076520 -0.062269      19        1  2.507809
18     0.363942 -0.038129      20        1  2.062867
14     0.030433 -0.033084      21        1  1.925519
7      0.200455  0.039971      22        1  1.487348
9      0.091851 -0.076437      23        1  1.342680
16     0.012920  0.023018      24        1  1.156448
4      0.043994 -0.116340      25        1  0.783022, topic_info=             Term          Freq         Total Category  logprob  loglift
71      repúblico  10358.000000  10358.000000  Default  30.0000  30.0000
79         social   6230.000000   6230.000000  Default  29.0000  29.0000
570      empresar   4112.000000   4112.000000  Default  28.0000  28.0000
1287       pontar   2243.000000   2243.000000  Default  27.0000  27.0000
684         cargo   4748.000000   4748.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
87           área     84.243615   4449.468529  Topic25  -4.7313   0.8829
172       direito     71.859605   3519.006988  Topic25  -4.8903   0.9585
500       limitar     59.054727   1075.084017  Topic25  -5.0865   1.9481
444   conservação     52.077320    686.381271  Topic25  -5.2123   2.2711
379        limite     48.942426   1414.499824  Topic25  -5.2744   1.4859

[1797 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
5476     20  0.961041     1
6196      9  0.929462  1011
3124     12  0.941159  1013
1024     12  0.992647  1014
3913     12  0.886451  1022
...     ...       ...   ...
1411      7  0.533649  útil
1411     11  0.128703  útil
1411     13  0.032961  útil
1411     14  0.034530  útil
1411     19  0.017265  útil

[6329 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 18, 14, 25, 22, 6, 24, 11, 21, 23, 4, 20, 1, 7, 16, 9, 12, 13, 2, 19, 15, 8, 10, 17, 5])